### Tesla Stock Price Prediction using Correlation and Random Forest Regression

Stock predictions are a common problem in finance. In this notebook, we will be using the stock price data of the S&P 500 company to predict the future stock prices. We will be using the Correlation to find the most important features and then using Random Forest Regression to predict the future stock prices. This work was done in google colab.


###

Data Collection and Preprocessing

We will be using the data from Yahoo Finance. To get this data we first use beautifulsoup to scape the S&P 500 tickers from wikipedia. We then use the yfinance package to get the stock data. We then use pandas to clean the data.
Data from the separate files are then merged into a single dataframe callded combined_df.
There are multiple data columns in the combined dataset after the merge so we drop all but the first and reorder the columns so that the target is the first column.

### 

Feature Extraction with Colleration

We find the correlations of the features with the target and take the first 10 values which have the strongest absolute correlations.

###

Creating the model
We split the data ino the training and testing set ,train the  and calculate the m2_score and the mean squared error.

### 

Conclusion
We had an r2_score of 0.99 and a mse of 52.76. This shows that our model did well since it had a high r2_score.  More optimizations can be made such as adding cross validation and making changes to the hyperparameters used. Also other models such as LSTMs, Artificial Neural Networks and Multi Layer Perceptrons can be used to seek better results.


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import bs4 as bs
import pickle
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,accuracy_score,mean_squared_error




Mounted at /content/drive


In [ ]:
html = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(html.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [ ]:
tickers = []
for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        ticker = ticker[:-1]
        tickers.append(ticker)

In [ ]:

with open("/content/drive/My Drive/sp500files/sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)

In [2]:
import datetime as dt
import os
import pandas_datareader.data as pdr

In [3]:
with open("/content/drive/My Drive/sp500files/sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
if not os.path.exists('/content/drive/My Drive/sp500files/stock_dfs'):
        os.makedirs('/content/drive/My Drive/sp500files/stock_dfs')

In [ ]:
start = dt.datetime(2000, 1, 1)
end = dt.datetime.now()

for ticker in tickers:
    if not os.path.exists('/content/drive/My Drive/sp500files/stock_dfs/{}.csv'.format(ticker)):
        df = pdr.DataReader(ticker.replace('.', '-'), 'yahoo', start, end)
        df.reset_index(inplace=True)
        df.set_index("Date", inplace=True)
        df.to_csv('/content/drive/My Drive/sp500files/stock_dfs/{}.csv'.format(ticker))
    else:
        print('Already have {}'.format(ticker) )

In [5]:
!pip install yfinance
import yfinance as yf

In [6]:

start = pd.to_datetime(['2014-01-01']).astype(int)[0]//10**9 # convert to unix timestamp.
end = pd.to_datetime(['2024-08-08']).astype(int)[0]//10**9 #onvert to unix timestamp.
count = 0
for ticker in tickers:
    url = 'https://query1.finance.yahoo.com/v7/finance/download/' +ticker.replace('.', '-')+'?period1='+str(start)+'&period2='+str(end)+'&interval=1d&events=history'
    # url = url.replace(" ", "%20")
    print(ticker)
    if not os.path.exists('/content/drive/My Drive/sp500files/stock_dfs/{}.csv'.format(ticker)):

        df = pd.read_csv(url)
        df.reset_index(inplace=True)
        df.set_index("Date", inplace=True)
        df.to_csv('/content/drive/My Drive/sp500files/stock_dfs/{}.csv'.format(ticker))


    else:
        print('Already have {}'.format(ticker))

        # print(len(tickers))

MMM
Already have MMM
AOS
Already have AOS
ABT
Already have ABT
ABBV
Already have ABBV
ACN
Already have ACN
ADBE
Already have ADBE
AMD
Already have AMD
AES
Already have AES
AFL
Already have AFL
A
Already have A
APD
Already have APD
ABNB
Already have ABNB
AKAM
Already have AKAM
ALB
Already have ALB
ARE
Already have ARE
ALGN
Already have ALGN
ALLE
Already have ALLE
LNT
Already have LNT
ALL
Already have ALL
GOOGL
Already have GOOGL
GOOG
Already have GOOG
MO
Already have MO
AMZN
Already have AMZN
AMCR
Already have AMCR
AEE
Already have AEE
AAL
Already have AAL
AEP
Already have AEP
AXP
Already have AXP
AIG
Already have AIG
AMT
Already have AMT
AWK
Already have AWK
AMP
Already have AMP
AME
Already have AME
AMGN
Already have AMGN
APH
Already have APH
ADI
Already have ADI
ANSS
Already have ANSS
AON
Already have AON
APA
Already have APA
AAPL
Already have AAPL
AMAT
Already have AMAT
APTV
Already have APTV
ACGL
Already have ACGL
ADM
Already have ADM
ANET
Already have ANET
AJG
Already have AJG
AIZ


In [35]:



# Create an empty DataFrame
combined_df = pd.DataFrame()

# Specify the directory containing the CSV files
csv_directory = '/content/drive/My Drive/sp500files/stock_dfs/'

# Iterate over the CSV files in the directory
for filename in os.listdir(csv_directory):
   if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(csv_directory, filename))

        # Extract the filename (without the extension) to use as the column name
        column_name = os.path.splitext(filename)[0]

        # Select the 'Date' and 'name' columns
        selected_columns = df[['Date', 'Close']]

        # Rename the 'name' column to match the file's column name
        selected_columns.rename(columns={'Close': column_name}, inplace=True)

        # Append the selected columns to the combined DataFrame
        combined_df = pd.concat([combined_df, selected_columns], axis=1)

# Print the combined DataFrame
print(combined_df)

<ipython-input-35-303ef10d7a8e>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.rename(columns={'Close': column_name}, inplace=True)
<ipython-input-35-303ef10d7a8e>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.rename(columns={'Close': column_name}, inplace=True)
<ipython-input-35-303ef10d7a8e>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.rename(columns={'Close': column_name

            Date         MMM        Date        AOS        Date         ABT  \
0     2014-01-02  115.493309  2014-01-02  26.660000  2014-01-02   38.230000   
1     2014-01-03  115.760872  2014-01-03  26.655001  2014-01-03   38.639999   
2     2014-01-06  115.075249  2014-01-06  26.635000  2014-01-06   39.150002   
3     2014-01-07  115.091972  2014-01-07  26.375000  2014-01-07   38.849998   
4     2014-01-08  114.239128  2014-01-08  26.075001  2014-01-08   39.200001   
...          ...         ...         ...        ...         ...         ...   
2575  2024-03-27   87.449837  2024-03-27  88.849998  2024-03-27  113.480003   
2576  2024-03-28   88.687294  2024-03-28  89.459999  2024-03-28  113.660004   
2577  2024-04-01   94.019997  2024-04-01  89.080002  2024-04-01  112.089996   
2578  2024-04-02   92.839996  2024-04-02  88.550003  2024-04-02  112.019997   
2579  2024-04-03   93.190002  2024-04-03  88.650002  2024-04-03  111.279999   

            Date        ABBV        Date         AC

In [36]:
# Drop all but the first 'Date' column
combined_df = combined_df.iloc[:, ~combined_df.columns.duplicated()]


In [37]:

# check for any NANs
combined_df.to_csv('/content/drive/My Drive/sp500files/combine_df.csv')
# combined_df.isna().count().sort_values(ascending=False)
#combined_df.dropna(inplace=True)

In [38]:
# Define the target column, which in this case is 'TSLA'
target = 'TSLA'

# Reorder the columns in the DataFrame so that the target column comes first.
column_titles =  [target] + [col for col in combined_df.columns if col != target]

# Reindex the DataFrame with the new column order.
# This moves the 'TSLA' column to the front and preserves the order of the other columns.
combined_df = combined_df.reindex(columns=column_titles)
combined_df.tail(5)



,TSLA,Date,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,...,WY,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
2575,179.830002,2024-03-27,87.449837,88.849998,113.480003,180.350006,340.940002,504.399994,179.589996,17.25,...,35.930000,38.540001,275.920013,101.790001,53.380001,129.380005,137.320007,299.160004,132.610001,168.509995
2576,175.789993,2024-03-28,88.687294,89.459999,113.660004,182.100006,346.609985,504.600006,180.490005,17.93,...,35.910000,38.970001,275.000000,102.230003,53.750000,129.240005,138.649994,301.440002,131.979996,169.210007
2577,175.220001,2024-04-01,94.019997,89.080002,112.089996,180.759995,339.170013,502.089996,183.339996,18.07,...,35.560001,38.869999,273.440002,106.559998,53.279999,128.639999,140.470001,301.000000,129.750000,167.020004
2578,166.630005,2024-04-02,92.839996,88.550003,112.019997,180.690002,336.459991,499.209991,178.699997,17.82,...,35.430000,39.299999,267.809998,105.260002,54.080002,128.119995,140.710007,296.500000,128.470001,165.009995
2579,168.380005,2024-04-03,93.190002,88.650002,111.279999,177.330002,332.279999,497.010010,180.770004,17.90,...,34.939999,39.349998,269.299988,106.419998,54.090000,128.860001,139.009995,292.750000,130.009995,162.970001


In [39]:
combined_df.set_index('Date',inplace=True)

In [40]:

corr = combined_df.corr()
corr

,TSLA,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,...,WY,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
TSLA,1.000000,-0.375396,0.705922,0.871964,0.811028,0.910701,0.843225,0.894105,0.805246,0.716370,...,0.430641,-0.136875,0.804939,-0.434107,0.730087,0.798157,0.794344,0.854145,0.482249,0.883881
MMM,-0.375396,1.000000,0.015517,-0.252834,-0.387145,-0.320914,-0.220054,-0.433798,-0.266570,-0.411362,...,0.183673,-0.278711,-0.282607,0.285850,-0.140911,-0.137901,-0.279462,-0.059079,0.158554,-0.284294
AOS,0.705922,0.015517,1.000000,0.746258,0.813442,0.847091,0.796836,0.784077,0.515233,0.805424,...,0.433134,-0.388254,0.799523,-0.324955,0.689416,0.863168,0.823619,0.692619,0.557881,0.794706
ABT,0.871964,-0.252834,0.746258,1.000000,0.820878,0.953591,0.948323,0.910352,0.854627,0.772741,...,0.207529,-0.344222,0.951804,-0.429322,0.911260,0.924984,0.927882,0.928390,0.670821,0.981448
ABBV,0.811028,-0.387145,0.813442,0.820878,1.000000,0.894114,0.776410,0.851934,0.680056,0.900657,...,0.345750,-0.150326,0.857225,-0.366778,0.737324,0.827324,0.872181,0.663296,0.403625,0.842794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,0.798157,-0.137901,0.863168,0.924984,0.827324,0.939803,0.928679,0.878443,0.795104,0.843524,...,0.259814,-0.388957,0.940480,-0.313050,0.849045,1.000000,0.947345,0.869880,0.693286,0.934190
YUM,0.794344,-0.279462,0.823619,0.927882,0.872181,0.945057,0.894429,0.876884,0.784579,0.914641,...,0.189874,-0.320269,0.950265,-0.370734,0.877072,0.947345,1.000000,0.812888,0.625608,0.941912
ZBRA,0.854145,-0.059079,0.692619,0.928390,0.663296,0.872479,0.891680,0.816291,0.853931,0.602700,...,0.328695,-0.323948,0.843789,-0.341660,0.810205,0.869880,0.812888,1.000000,0.703694,0.907078
ZBH,0.482249,0.158554,0.557881,0.670821,0.403625,0.586614,0.656594,0.533082,0.642833,0.430670,...,0.241779,-0.414552,0.679267,-0.171115,0.676749,0.693286,0.625608,0.703694,1.000000,0.644456


In [12]:
# The correlation heatmap
from statsmodels.graphics.correlation import plot_corr
import matplotlib.pyplot as plt

# Plot the correlation matrix (This isn't helpful in this case since we have a lot of columns)
# fig = plt.figure(figsize=(15,15))
# ax = fig.add_subplot(111)
# plot_corr(corr, xnames = corr.columns, ax = ax)


In [13]:
# Calculate the absolute correlation values for the target column
corr[target].abs().sort_values(ascending=False)

,TSLA
TSLA,1.000000
SOLV,0.963689
GEV,0.960767
WST,0.937383
AAPL,0.935649
...,...
PNW,0.024014
TRGP,0.022391
ABNB,0.014459
XOM,0.011466


In [14]:

# Sort the correlation values in descending order and select the top 10
top_corr_values = corr[target].abs().sort_values(ascending=False).head(13)
top_corr_values

,TSLA
TSLA,1.000000
SOLV,0.963689
GEV,0.960767
WST,0.937383
AAPL,0.935649
DHR,0.929809
BX,0.928206
POOL,0.927776
LOW,0.927389
UPS,0.925320


In [15]:
''' Exclude the first two correlation values from the dataset.
 This is because the first is the target which will always have a full correlation with itself 
 and the second joined the S&P500 much later and doesnt have as many values.'''
X = combined_df[top_corr_values.index[3:]]
y = combined_df[target]
# Display the first few rows of the adjusted dataset
X.head(100)

,WST,AAPL,DHR,BX,POOL,LOW,UPS,TMO,GOOGL,GOOG
Date,,,,,,,,,,
2014-01-02,48.680000,19.754642,51.383492,31.280001,57.360001,49.439999,103.320000,109.629997,27.855856,27.724083
2014-01-03,49.119999,19.320715,51.457424,32.049999,57.340000,48.950001,103.089996,110.050003,27.652653,27.521841
2014-01-06,48.849998,19.426071,51.007103,31.480000,56.570000,48.259998,101.750000,110.059998,27.960960,27.828691
2014-01-07,49.220001,19.287144,51.430538,31.750000,56.860001,48.380001,102.080002,112.309998,28.500000,28.365179
2014-01-08,49.930000,19.409286,51.457424,31.520000,56.700001,48.549999,102.000000,114.040001,28.559309,28.424210
...,...,...,...,...,...,...,...,...,...,...
2014-05-20,41.700001,21.596786,50.274494,30.010000,57.189999,45.520000,100.529999,114.470001,27.019501,26.415976
2014-05-21,42.259998,21.653929,51.269230,31.000000,57.560001,45.410000,101.739998,115.320000,27.485001,26.873220
2014-05-22,42.209999,21.688213,52.445438,31.260000,57.939999,46.889999,101.660004,115.930000,27.772499,27.178383


In [41]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:


# Initialise the RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model
rf_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_regressor.predict(X_test)

# Calculate R-squared
r_squared = r2_score(y_test, y_pred)

print(f"R-squared of the RandomForestRegressor: {r_squared}")

# 5. Calculate Mean Squared Error (MSE) as an additional metric
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")


R-squared of the RandomForestRegressor: 0.9952113363173738
Mean Squared Error (MSE): 52.76747334367679
